# Introduction to Opik Agent Optimizers

You will need:

1. A Comet account, for seeing Opik visualizations (free!) - [comet.com](https://comet.com)
2. An OpenAI account, for using an LLM
[platform.openai.com/settings/organization/api-keys](https://platform.openai.com/settings/organization/api-keys)


## Setup

This pip-install takes about a minute.

In [1]:
%%capture
%pip install opik-optimizer

Let's make sure we have the correct version:

In [2]:
import opik_optimizer
opik_optimizer.__version__

'0.8.1'

The version should be 0.7.3 or greater.

[Comet](https://www.comet.com/site?from=llm&utm_source=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key.

> You can also run the Opik platform locally, see the [installation guide](https://www.comet.com/docs/opik/self-host/overview/?from=llm&utm_source=opik) for more information.

Enter your Comet API key, followed by "Y" to use your own workspace:

In [3]:
import opik

# Configure Opik
opik.configure()

OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/jacquesverre/.opik.config


For this example, we'll use OpenAI models, so we need to set our OpenAI API key:

In [4]:
import os
import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

To optimize any prompt, we'll need three basic things:

1. A starting prompt
2. A metric
3. A dataset

## The Dataset

In this experiment, we are going to use the **HotPotQA** dataset. This dataset was designed to be difficult for regular LLMs to handle. This dataset is called a "**multi-hop**" dataset because answering the questions involves multiple reasoning steps and multiple tool calls, where the LLM needs to infer relationships, combine information, or draw conclusions based on the combined context.

Example:

> "What are the capitals of the states that border California?"

You'd need to find which states border California, and then lookup each state's capital.

The dataset has about 113,000 such crowd-sourced questions that are constructed to require the introductory paragraphs of two Wikipedia articles to answer.

**NOTE:** The name "HotPot" comes from the restaurant where the authors came up with the idea of the dataset.

In [5]:
import opik_optimizer

opik_dataset = opik_optimizer.datasets.hotpot_300()

Let's take a look at some dataset items:

In [6]:
rows = opik_dataset.get_items()
rows[0]

{'id': '0197044c-d782-7735-a8cf-bffd4d19838e',
 'question': 'Are Smyrnium and Nymania both types of plant?',
 'answer': 'yes'}

We see that each item has a "question" and "answer". Some of the answers are short and direct, and some of them are more complicated:

In [7]:
rows[1]

{'id': '0197044c-d781-7eb0-a717-6fd6b1415476',
 'question': 'That Darn Cat! and Never a Dull Moment were both produced by what studio?',
 'answer': 'Walt Disney Productions'}

## Opik Project

All LLM traces in Opik are saved in a "project". We'll put them all in the following project name:

In [8]:
project_name = "optimize-workshop-2025"

## The Metric

Choosing a good metric for optimization is tricky. For these examples, we'll pick one that will allow us to show improvement, and also provide a gradient of scores. In general though, this metric isn't the best for optimization runs.

We'll use "Edit Distance" AKA "Levenshtein Distance":

In [9]:
from opik.evaluation.metrics import LevenshteinRatio

metric = LevenshteinRatio(project_name=project_name)

The metric takes two things: the `output` of the LLM and the `reference` (the truth):

In [10]:
metric.score("Hello", "Hello")

OPIK: Started logging traces to the "optimize-workshop-2025" project at https://www.comet.com/opik/api/v1/session/redirect/projects/?trace_id=019740d5-afcf-7e3d-a2ec-bd2e4a759aa8&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==.


ScoreResult(name='levenshtein_ratio_metric', value=1.0, reason=None, metadata=None, scoring_failed=False)

In [11]:
metric.score("Hello!", "Hello")

ScoreResult(name='levenshtein_ratio_metric', value=0.9090909090909091, reason=None, metadata=None, scoring_failed=False)

The edit distance between "Hello!" and "Hello" is 1. Here is how the .91 is computed:

In [12]:
edit_distance = 1

1 - edit_distance / (len("Hello1") + len("Hello"))


0.9090909090909091

For more information see: [Levenshtein Distance](https://en.wikipedia.org/wiki/Levenshtein_distance)

## Configuation

To create the necessary configurations for using an Opik Agent Optimizer, you'll need two things:

1. An initial prompt
2. A metric

We're going to start with a pretty bad prompt... so we can optimize it!

In [13]:
from opik_optimizer import ChatPrompt

initial_prompt = ChatPrompt(
    messages=[
        {"role": "system", "content": "Provide an answer to the question"},
        {"role": "user", "content": "{question}"}
    ]
)

And the metric:

In [14]:
def levenshtein_ratio(dataset_item, llm_output):
    metric = LevenshteinRatio()
    return metric.score(reference=dataset_item['answer'], output=llm_output)

As you can see our metric function is a wrapper around the built-in LevenshteinRatio(). For most Opik metrics, we need two parameters.
1. 'output' - which is the output of the LLM
2. 'reference' - the correct answer (provided by the database item "answer")



## FewShotBayesianOptimizer

The FewShotBayesianOptimizer name indicates two things:

1. It will produce Chat Prompts, or FewShot examples as described in the slides.
2. Secondly, it describes how it searches for the best set of these FewShot examples.

To use this optimizer, we import it and create an instance, passing in the project name and model parameters:

In [15]:
from opik_optimizer import (
    FewShotBayesianOptimizer,
)

optimizer = FewShotBayesianOptimizer(
    project_name=project_name,
    model="openai/gpt-4o-mini",
    temperature=0.1,
    max_tokens=5000,
)

### Baseline

Before we optimize this prompt ("Provide an answer to the question") let's see what the bare prompt does by itself on the dataset:

In [16]:
score = optimizer.evaluate_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    n_samples=100,
)
score

Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]

╭─ hotpot_300 (100 samples) ─────────────╮
│                                        │
│ Total time:        00:00:13            │
│ Number of samples: 100                 │
│                                        │
│ levenshtein_ratio_metric: 0.1375 (avg) │
│                                        │
╰────────────────────────────────────────╯

Uploading results to Opik ...

View the results ]8;id=324753;https://www.comet.com/opik/api/v1/session/redirect/experiments/?experiment_id=019740d5-b273-7247-901e-0230ea043eb6&dataset_id=0197044c-d51d-7cff-96c5-38969b6ae708&path=aHR0cHM6Ly93d3cuY29tZXQuY29tL29waWsvYXBpLw==\in your Opik dashboard]8;;\.

0.13748748158828172

In my run, it scored about 16% correct. [I say "percent correct" but because we are using edit distance, that isn't quite accurate. But we can think of it this way.] Your result may be somewhat different but is probably between 10% and 20% correct.

Ok, let's optimize that prompt!

In [17]:
result = optimizer.optimize_prompt(
    prompt=initial_prompt,
    dataset=opik_dataset,
    metric=levenshtein_ratio,
    n_trials=3,
    n_samples=50
)

╭────────────────────────────────────────────────────────────────────╮
│ ● Running Opik Evaluation - Few-Shot Bayesian Optimizer            │
╰────────────────────────────────────────────────────────────────────╯

> Let's optimize the prompt:

╭─ system ───────────────────────────────────────────────────────────╮
│                                                                    │
│  Provide an answer to the question                                 │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯
╭─ user ─────────────────────────────────────────────────────────────╮
│                                                                    │
│  {question}                                                        │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯


Using FewShotBayesianOptimizer with the parameters:

  - metric: levenshtein_ratio

  - n_trials: 3

  - n_samples: 50

> First we will establish the baseline performance:
  Baseline score was: 0.1486.

> Let's add a placeholder for few-shot examples in the messages:
│    Created the prompt template:
│
│    ╭─ system ───────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  Provide an answer to the question                                 │
│    │                                                                    │
│    │  ### Few-Shot Examples                                             │
│    │  FEW_SHOT_EXAMPLE_PLACEHOLDER                                      │
│    │                                                                    │
│    ╰────────────────────────────────────────────────────────────────────╯
│    ╭─ user ─────────────────────────────────────────────────────────────╮
│    │                                                                    │
│    │  {question}                                      

> Optimization complete

╭─ Optimization results ─────────────────────────────────────────────╮
│                                                                    │
│  Prompt was optimized and improved from 0.1486 to 0.3669           │
│  (146.80%)                                                         │
│                                                                    │
│  Optimized prompt:                                                 │
│  ╭─ system ─────────────────────────────────────────────────────╮  │
│  │                                                              │  │
│  │  Provide an answer to the question                           │  │
│  │                                                              │  │
│  │  ### Few-Shot Examples                                       │  │
│  │  Question: What novel did an author who was featured in the  │  │
│  │  Voices of Ghana and later became a journalist write?        │  │
│  │  Answer: The Gab Boys                                        │  │
│  │                                                              │  │
│  │  Question: What were the dates of the campaign that          │  │
│  │  eventually led to the Treaty of Roskilde?                   │  │
│  │  Answer: between 30 January and 8 February 1658              │  │
│  │                                                              │  │
│  │  Question: Are Wanding Town and Jiujiang, Guangdong both     │  │
│  │  frontier towns?                                             │  │
│  │  Answer: no                                                  │  │
│  │                                                              │  │
│  │  Question: The Knicks–Nuggets was the most penalized         │  │
│  │  on-court fight in the NBA since an altercation that         │  │
│  │  occured in what city?                                       │  │
│  │  Answer: Auburn Hills                                        │  │
│  │                                                              │  │
│  ╰──────────────────────────────────────────────────────────────╯  │
│  ╭─ user ───────────────────────────────────────────────────────╮  │
│  │                                                              │  │
│  │  {question}                                                  │  │
│  │                                                              │  │
│  ╰──────────────────────────────────────────────────────────────╯  │
│                                                                    │
╰────────────────────────────────────────────────────────────────────╯

In [18]:
result.display()

╔═════════════════════════════════════════════ Optimization Complete ═════════════════════════════════════════════╗
║                                                                                                                 ║
║ Optimizer:         FewShotBayesianOptimizer                                                                     ║
║ Model Used:        openai/gpt-4o-mini (Temp: 0.1)                                                               ║
║ Metric Evaluated:  levenshtein_ratio                                                                            ║
║ Initial Score:     N/A                                                                                          ║
║ Final Best Score:  0.3669                                                                                       ║
║ Total Improvement: N/A (no initial score)                                                                       ║
║ Rounds Completed:  0                                                                                            ║
║ Stopped Early:     False                                                                                        ║
║                                                                                                                 ║
║                                                                                                                 ║
║ ╭────────────────────────────────────────── Final Optimized Prompt ───────────────────────────────────────────╮ ║
║ │                                                                                                             │ ║
║ │  System: Provide an answer to the question                                                                  │ ║
║ │                                                                                                             │ ║
║ │  ### Few-Shot Examples                                                                                      │ ║
║ │  Question: What novel did an author who was featured in the Voices of Ghana and later became a journalist   │ ║
║ │  write?                                                                                                     │ ║
║ │  Answer: The Gab Boys                                                                                       │ ║
║ │                                                                                                             │ ║
║ │  Question: What were the dates of the campaign that eventually led to the Treaty of Roskilde?               │ ║
║ │  Answer: between 30 January and 8 February 1658                                                             │ ║
║ │                                                                                                             │ ║
║ │  Question: Are Wanding Town and Jiujiang, Guangdong both frontier towns?                                    │ ║
║ │  Answer: no                                                                                                 │ ║
║ │                                                                                                             │ ║
║ │  Question: The Knicks–Nuggets was the most penalized on-court fight in the NBA since an altercation that    │ ║
║ │  occured in what city?                                                                                      │ ║
║ │  Answer: Auburn Hills                                                                                       │ ║
║ │  ---                                                                                                        │ ║
║ │  User: {question}                                                                                           │ ║
║ │  ---                                                                                                        │ ║
║ │                                                                                                             │ ║
║ ╰─────────────────────────────────────────────────────

Well done optimizer!

The percentage correct went from about 15% to about 50% correct.

What did we find? The result is a series of messages:

In [25]:
result.prompt

[{'role': 'system',
  'content': 'Provide an answer to the question\n\n### Few-Shot Examples\nQuestion: What novel did an author who was featured in the Voices of Ghana and later became a journalist write?\nAnswer: The Gab Boys\n\nQuestion: What were the dates of the campaign that eventually led to the Treaty of Roskilde?\nAnswer: between 30 January and 8 February 1658\n\nQuestion: Are Wanding Town and Jiujiang, Guangdong both frontier towns?\nAnswer: no\n\nQuestion: The Knicks–Nuggets was the most penalized on-court fight in the NBA since an altercation that occured in what city?\nAnswer: Auburn Hills'},
 {'role': 'user', 'content': '{question}'}]

## Opik Visualization UI

When you create an Optimization Run, you'll see it in the Opik UI (either running locally or hosted).

If you go to your `comet.com/opik/YOURID/optimizations` page, you'll see your run at the top of the list:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimizer-ui.png" width="500px">

Along the top of the page you'll see a running history of the metric scores, with the latest dataset selected. 

If you click on your run, you'll see the set of trials that ran durring this optimization run. Running across the top of this page are the scores just for this trial. On the top right you'll see the best so far:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimize-trials.png" width="500px">

If you click on a trial, you'll see the prompt for that trial:

<img src="https://raw.githubusercontent.com/comet-ml/opik/refs/heads/main/sdks/opik_optimizer/docs/images/optimizer-prompt.png" width="500px">

From the trial page you can also see the Trial items, and even dig down (mouse over the "Evaluation task" column on a row) to see the traces.

## Using Optimized Prompts

How can we use the optimized results?


Once we have the "chat_messages", we can do the following:

In [20]:
from litellm.integrations.opik.opik import OpikLogger
import litellm
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

def query(question, chat_messages):
    messages = chat_messages[:-1] # Cut off the last one
    # replace it with question in proper format:
    messages.append({'role': 'user', 'content': '{"question": "%s"}"}' % question})

    response = litellm.completion(
        model="gpt-4o-mini",
        temperature=0.1,
        max_tokens=5000,
        messages=messages,
    )
    return response.choices[0].message.content

In [21]:
query("When was David Chalmers born?", result.details["chat_messages"])

'Answer: April 20, 1966'

In [22]:
query("What weighs more: a baby elephant or an SUV?", result.details["chat_messages"])

'Answer: a baby elephant'

If it says "elephant" that is not correct!

We'll need to use an agent with tools to get a better answer.

# Next Steps

You can try out other optimizers. More details can be found in the [Opik Agent Optimizer documentation](https://www.comet.com/docs/opik/agent_optimization/overview).